# MultiAgent Pmp

#### Load Relevant Libraries

In [1]:
# pip install pypdf
# pip install langchain_openai
# pip install langchain_chroma
# pip install langchain_text_splitters
# pip install langchain_core
# pip install langchain
# pip install langchain_community

In [2]:
import os
import pandas as pd
from pypdf import PdfReader
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools.tavily_search import TavilySearchResults
import json
from datetime import datetime, timedelta
from typing import Dict, Any, List, Optional
import random

/Users/christopherwaight/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Set up Logging

In [3]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

### Logging into Open AI

In [4]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")


from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

## Setting up websearch tools

In [5]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

### Loading Project Documents

In [6]:
import os
import glob
from pypdf import PdfReader
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def load_project_documents():
    """Load all project-specific text documents"""
    documents = []
    
    # Load all .txt files (project documents)
    txt_files = glob.glob("*.txt")
    for txt_file in txt_files:
        print(f"Loading project document: {txt_file}")
        try:
            with open(txt_file, 'r', encoding='utf-8') as f:
                text = f.read()
                doc = Document(
                    page_content=text,
                    metadata={
                        "source": txt_file, 
                        "type": "project_document",
                        "category": "fence_project"
                    }
                )
                documents.append(doc)
        except Exception as e:
            print(f"Error loading {txt_file}: {e}")
    
    return documents

### Loading PMBOK

In [7]:
def load_pmbok_documents():
    """Load PMBOK PDF files"""
    documents = []
    
    # Load PMBOK PDF files
    pdf_files = ["PMBOK7.pdf", "pmbokinfographic.pdf"]
    for pdf_file in pdf_files:
        if os.path.exists(pdf_file):
            print(f"Loading PMBOK document: {pdf_file}")
            try:
                reader = PdfReader(pdf_file)
                for page_num, page in enumerate(reader.pages):
                    text = page.extract_text()
                    if text and text.strip():
                        doc = Document(
                            page_content=text,
                            metadata={
                                "source": pdf_file, 
                                "page": page_num + 1, 
                                "type": "pmbok",
                                "category": "best_practices"
                            }
                        )
                        documents.append(doc)
            except Exception as e:
                print(f"Error loading {pdf_file}: {e}")
        else:
            print(f"Warning: {pdf_file} not found")
    
    return documents

### SetUp Dual Vector Stores

In [8]:
# Create separate vector stores
def setup_dual_vector_stores():
    """Set up separate vector stores for project docs and PMBOK"""
    
    # Load documents separately
    print("Loading project documents...")
    project_docs = load_project_documents()
    print(f"Loaded {len(project_docs)} project documents\n")
    
    print("Loading PMBOK documents...")
    pmbok_docs = load_pmbok_documents()
    print(f"Loaded {len(pmbok_docs)} PMBOK pages\n")
    
    # Create embeddings (same for both)
    embeddings = OpenAIEmbeddings()
    
    # Split project documents (larger chunks for structured docs)
    project_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=300,
        add_start_index=True
    )
    project_splits = project_splitter.split_documents(project_docs)
    print(f"Created {len(project_splits)} project document chunks")
    
    # Split PMBOK documents (smaller chunks for dense content)
    pmbok_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        add_start_index=True
    )
    pmbok_splits = pmbok_splitter.split_documents(pmbok_docs)
    print(f"Created {len(pmbok_splits)} PMBOK chunks\n")
    
    # Create separate vector stores
    print("Creating project vector store...")
    project_vectorstore = Chroma.from_documents(
        documents=project_splits,
        embedding=embeddings,
        collection_name="fence_project_documents"
    )
    
    print("Creating PMBOK vector store...")
    pmbok_vectorstore = Chroma.from_documents(
        documents=pmbok_splits,
        embedding=embeddings,
        collection_name="pmbok_best_practices"
    )
    
    # Create retrievers
    project_retriever = project_vectorstore.as_retriever(search_kwargs={"k": 5})
    pmbok_retriever = pmbok_vectorstore.as_retriever(search_kwargs={"k": 3})
    
    return project_retriever, pmbok_retriever, project_vectorstore, pmbok_vectorstore

# Set up the stores
project_retriever, pmbok_retriever, project_vectorstore, pmbok_vectorstore = setup_dual_vector_stores()

Loading project documents...
Loading project document: Stakeholder Register.txt
Loading project document: ReportTemplates.txt
Loading project document: Baseline Schedule.txt
Loading project document: ChangeScenarios.txt
Loading project document: Initial_Budget.txt
Loading project document: WorkBreakDownStructure.txt
Loading project document: Project Charter.txt
Loading project document: riskRegister.txt
Loaded 8 project documents

Loading PMBOK documents...
Loading PMBOK document: PMBOK7.pdf
Loading PMBOK document: pmbokinfographic.pdf
Loaded 371 PMBOK pages

Created 22 project document chunks
Created 935 PMBOK chunks

Creating project vector store...
Creating PMBOK vector store...


### Testing that RAG is working properly

In [9]:


# Q&A Functions for different use cases
def ask_project_specific(question, llm):
    """Ask questions about the specific project only"""
    
    prompt = ChatPromptTemplate.from_template("""
    Based on the project documents, answer this question about the Backyard Fence Replacement project.
    
    Context: {context}
    Question: {question}
    
    Provide specific details from the project documents only.
    Answer:
    """)
    
    docs = project_retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"question": question, "context": context})
    
    sources = list(set([doc.metadata.get('source', 'Unknown') for doc in docs]))
    return answer, sources

def ask_pmbok_best_practices(question, llm):
    """Ask questions about PMBOK best practices"""
    
    prompt = ChatPromptTemplate.from_template("""
    Based on PMBOK best practices, answer this project management question.
    
    Context: {context}
    Question: {question}
    
    Provide guidance based on PMBOK standards and best practices.
    Answer:
    """)
    
    docs = pmbok_retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"question": question, "context": context})
    
    sources = list(set([doc.metadata.get('source', 'Unknown') for doc in docs]))
    return answer, sources

def ask_combined(question, llm):
    """Ask questions using both project docs and PMBOK"""
    
    prompt = ChatPromptTemplate.from_template("""
    Answer this question using both the specific project information and PMBOK best practices.
    
    Project Context:
    {project_context}
    
    PMBOK Best Practices:
    {pmbok_context}
    
    Question: {question}
    
    First provide the specific answer from the project, then explain how it aligns with or differs from PMBOK best practices.
    Answer:
    """)
    
    # Get docs from both stores
    project_docs = project_retriever.invoke(question)
    pmbok_docs = pmbok_retriever.invoke(question)
    
    project_context = "\n\n".join([doc.page_content for doc in project_docs])
    pmbok_context = "\n\n".join([doc.page_content for doc in pmbok_docs])
    
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({
        "question": question, 
        "project_context": project_context,
        "pmbok_context": pmbok_context
    })
    
    project_sources = list(set([doc.metadata.get('source', 'Unknown') for doc in project_docs]))
    pmbok_sources = list(set([doc.metadata.get('source', 'Unknown') for doc in pmbok_docs]))
    
    return answer, {"project": project_sources, "pmbok": pmbok_sources}

# Example usage
print("=== TESTING DUAL VECTOR STORES ===\n")

# Test project-specific query
question1 = "What is the budget for the fence project?"
answer, sources = ask_project_specific(question1, llm)
print(f"Project Query: {question1}")
print(f"Answer: {answer}")
print(f"Sources: {sources}\n")

# Test PMBOK query
question2 = "What are the key components of a project charter?"
answer, sources = ask_pmbok_best_practices(question2, llm)
print(f"PMBOK Query: {question2}")
print(f"Answer: {answer}")
print(f"Sources: {sources}\n")

# Test combined query
question3 = "Does the fence project charter follow PMBOK best practices?"
answer, sources = ask_combined(question3, llm)
print(f"Combined Query: {question3}")
print(f"Answer: {answer}")
print(f"Project Sources: {sources['project']}")
print(f"PMBOK Sources: {sources['pmbok']}")

=== TESTING DUAL VECTOR STORES ===

Project Query: What is the budget for the fence project?
Answer: The budget for the Backyard Fence Replacement project is $4,500, which includes a 10% contingency. This total budget is detailed in the project documents as encompassing all expected project costs.
Sources: ['riskRegister.txt', 'ReportTemplates.txt', 'Project Charter.txt', 'Initial_Budget.txt']

PMBOK Query: What are the key components of a project charter?
Answer: The project charter is a critical document in project management, as it formally authorizes the project and provides key details necessary for guiding project initiation. According to PMBOK (Project Management Body of Knowledge) standards and best practices, the key components of a project charter generally include the following:

1. **Project Purpose or Justification**: This section explains the reason the project is being undertaken. It outlines the business need that the project addresses and provides the rationale, detail

### Load Agent Behaviour from Text Files

In [23]:
# Load agent roles from text files
def load_agent_role(filename: str) -> str:
    """Load agent role from text file"""
    try:
        with open(f'agents/{filename}', 'r') as f:
            return f.read()
    except FileNotFoundError:
        print(f"Warning: {filename} not found. Using default role.")
        return f"You are a {filename.replace('_', ' ').replace('.txt', '')}."

# Load all agent roles
OBSERVER_ROLE = load_agent_role('observer_agent_role.txt')
RISK_ANALYZER_ROLE = load_agent_role('risk_analyzer_role.txt')
IMPACT_CALCULATOR_ROLE = load_agent_role('impact_calculator_role.txt')
WEB_RESEARCHER_ROLE = load_agent_role('web_researcher_role.txt')
DECISION_CONTROLLER_ROLE = load_agent_role('decision_controller_role.txt')
REPORT_WRITER_ROLE = load_agent_role('report_writer_role.txt')

### Creating Agent Classes

In [24]:
class ReportWriter:
    """Generates appropriate emails based on decisions"""
    
    def __init__(self, llm, retriever):
        self.llm = llm
        self.retriever = retriever
        
    def write_report(self, decision: Dict, all_data: Dict) -> Dict[str, Any]:
        print(f"\n[Report Writer] Generating {decision['communication_type']}...")
        
        # Get report templates
        template_docs = self.retriever.invoke("report templates email")
        templates = "\n\n".join([doc.page_content for doc in template_docs])
        
        prompt = ChatPromptTemplate.from_template(
            REPORT_WRITER_ROLE +
            "\n\nReport Templates:\n{templates}\n\n" +
            "Decision: {decision}\n" +
            "Complete Data: {data}\n\n" +
            "Generate the appropriate email in JSON format:"
        )
        
        chain = prompt | self.llm | StrOutputParser()
        result = chain.invoke({
            "templates": templates,
            "decision": json.dumps(decision),
            "data": json.dumps(all_data)
        })
        
        try:
            # Clean the response before parsing
            cleaned_result = clean_json_response(result)
            parsed = json.loads(cleaned_result)
            print(f"[Report Writer] Generated {parsed['email_type']} email")
            return parsed
        except json.JSONDecodeError as e:
            print(f"[Report Writer] JSON parse error: {str(e)}")
            print(f"[Report Writer] Raw result: {result[:200]}...")
            return {
                "email_type": "ERROR",
                "subject": "Error generating report",
                "body": "An error occurred while generating the report."
            }

# Updated DecisionController class with fix
class DecisionController:
    """Makes escalation decisions and creates action plans"""
    
    def __init__(self, llm, retriever):
        self.llm = llm
        self.retriever = retriever
        
    def decide(self, situation_data: Dict, risk_analysis: Dict, 
               impact_assessment: Dict, web_research: Optional[Dict] = None) -> Dict[str, Any]:
        print("\n[Decision Controller] Evaluating escalation requirements...")
        
        # Get escalation thresholds
        threshold_docs = self.retriever.invoke("escalation thresholds approval")
        context = "\n\n".join([doc.page_content for doc in threshold_docs])
        
        # Compile all information
        all_data = {
            "situation": situation_data,
            "risk_analysis": risk_analysis,
            "impact_assessment": impact_assessment,
            "web_research": web_research if web_research is not None else {"not_needed": True}
        }
        
        prompt = ChatPromptTemplate.from_template(
            DECISION_CONTROLLER_ROLE +
            "\n\nProject Thresholds:\n{context}\n\n" +
            "Complete Analysis:\n{analysis}\n\n" +
            "Make decision and provide response in JSON format:"
        )
        
        chain = prompt | self.llm | StrOutputParser()
        result = chain.invoke({
            "context": context,
            "analysis": json.dumps(all_data, indent=2)
        })
        
        try:
            # Clean the response before parsing
            cleaned_result = clean_json_response(result)
            parsed = json.loads(cleaned_result)
            print(f"[Decision Controller] Escalation required: {parsed['escalation_required']}")
            if parsed['escalation_required']:
                print(f"[Decision Controller] Reason: {parsed['escalation_reason']}")
            return parsed
        except json.JSONDecodeError as e:
            print(f"[Decision Controller] JSON parse error: {str(e)}")
            print(f"[Decision Controller] Raw result: {result[:200]}...")
            return {
                "escalation_required": True,
                "escalation_reason": "Unable to process - defaulting to escalation",
                "communication_type": "ESCALATION_EMAIL"
            }

# Apply the same fix to other agents that might have JSON parsing issues
class RiskAnalyzer:
    """Analyzes situations against risk register"""
    
    def __init__(self, llm, retriever):
        self.llm = llm
        self.retriever = retriever
        
    def analyze(self, situation_data: Dict[str, Any]) -> Dict[str, Any]:
        print("\n[Risk Analyzer] Checking against risk register...")
        
        # Search risk register
        risk_docs = self.retriever.invoke(f"risk {situation_data['situation']}")
        risk_context = "\n\n".join([doc.page_content for doc in risk_docs])
        
        prompt = ChatPromptTemplate.from_template(
            RISK_ANALYZER_ROLE + 
            "\n\nRisk Register Context:\n{context}\n\n" +
            "Situation to analyze:\n{situation}\n\n" +
            "Provide your analysis in JSON format:"
        )
        
        chain = prompt | self.llm | StrOutputParser()
        result = chain.invoke({
            "context": risk_context,
            "situation": json.dumps(situation_data)
        })
        
        try:
            cleaned_result = clean_json_response(result)
            parsed = json.loads(cleaned_result)
            print(f"[Risk Analyzer] Classification: {parsed['classification']}")
            if parsed.get('risk_id'):
                print(f"[Risk Analyzer] Matched Risk ID: {parsed['risk_id']}")
            return parsed
        except json.JSONDecodeError:
            print("[Risk Analyzer] Parse error, returning unknown risk")
            return {
                "classification": "UNKNOWN_RISK",
                "risk_id": None,
                "documented_mitigation": None,
                "confidence": "LOW"
            }

# Apply fix to all other agents similarly...
class ObserverAgent:
    """Parses input and extracts structured information"""
    
    def __init__(self, llm):
        self.llm = llm
        self.prompt = ChatPromptTemplate.from_template(
            OBSERVER_ROLE + "\n\nInput: {input}\n\nProvide your analysis in JSON format:"
        )
        
    def process(self, user_input: str) -> Dict[str, Any]:
        print("\n[Observer Agent] Parsing input...")
        chain = self.prompt | self.llm | StrOutputParser()
        result = chain.invoke({"input": user_input})
        
        try:
            cleaned_result = clean_json_response(result)
            parsed = json.loads(cleaned_result)
            print(f"[Observer Agent] Extracted: Date={parsed['date']}, Type={parsed['situation_type']}")
            return parsed
        except json.JSONDecodeError:
            print("[Observer Agent] Failed to parse JSON, using fallback")
            # Fallback parsing
            parts = user_input.split(" - ", 1)
            return {
                "date": parts[0] if len(parts) > 0 else "Unknown",
                "situation": parts[1] if len(parts) > 1 else user_input,
                "entities_affected": [],
                "situation_type": "OTHER",
                "duration_mentioned": None
            }

class ImpactCalculator:
    """Calculates schedule and cost impacts"""
    
    def __init__(self, llm, retriever):
        self.llm = llm
        self.retriever = retriever
        
    def calculate(self, situation_data: Dict, risk_analysis: Dict) -> Dict[str, Any]:
        print("\n[Impact Calculator] Assessing project impact...")
        
        # Get project schedule and budget context
        schedule_docs = self.retriever.invoke("schedule baseline milestones budget")
        context = "\n\n".join([doc.page_content for doc in schedule_docs])
        
        prompt = ChatPromptTemplate.from_template(
            IMPACT_CALCULATOR_ROLE +
            "\n\nProject Context:\n{context}\n\n" +
            "Situation: {situation}\n" +
            "Risk Analysis: {risk_analysis}\n\n" +
            "Calculate impacts and provide response in JSON format:"
        )
        
        chain = prompt | self.llm | StrOutputParser()
        result = chain.invoke({
            "context": context,
            "situation": json.dumps(situation_data),
            "risk_analysis": json.dumps(risk_analysis)
        })
        
        try:
            cleaned_result = clean_json_response(result)
            parsed = json.loads(cleaned_result)
            print(f"[Impact Calculator] Schedule impact: {parsed['schedule_impact_days']} days")
            print(f"[Impact Calculator] Cost impact: ${parsed['cost_impact_dollars']}")
            print(f"[Impact Calculator] Web research needed: {parsed['needs_web_research']}")
            return parsed
        except json.JSONDecodeError:
            return {
                "schedule_impact_days": 0,
                "cost_impact_dollars": 0,
                "needs_web_research": False,
                "confidence": "LOW"
            }

class WebResearcher:
    """Conducts web research when needed"""
    
    def __init__(self, llm):
        self.llm = llm
        self.search_tool = TavilySearchResults(max_results=3)
        
    def research(self, query: str, location: str = "San Jose, California") -> Dict[str, Any]:
        print(f"\n[Web Researcher] Searching for: {query}")
        
        try:
            # Add location context to query
            search_query = f"{query} {location} 2025 prices"
            search_results = self.search_tool.invoke(search_query)
            
            # Process results with LLM
            prompt = ChatPromptTemplate.from_template(
                WEB_RESEARCHER_ROLE +
                "\n\nSearch Query: {query}\n" +
                "Search Results: {results}\n\n" +
                "Analyze and provide findings in JSON format:"
            )
            
            chain = prompt | self.llm | StrOutputParser()
            result = chain.invoke({
                "query": search_query,
                "results": str(search_results)
            })
            
            cleaned_result = clean_json_response(result)
            parsed = json.loads(cleaned_result)
            print(f"[Web Researcher] Found: {parsed.get('findings', {}).get('primary_option', {}).get('description', 'No results')}")
            return parsed
            
        except Exception as e:
            print(f"[Web Researcher] Search failed: {str(e)}")
            return {
                "search_successful": False,
                "error": str(e),
                "fallback_estimate": "Unable to determine - recommend escalation"
            }
        
# Main Orchestrator
class ProjectManagementMAS:
    """Orchestrates all agents in sequence"""
    
    def __init__(self, llm, project_retriever):
        self.observer = ObserverAgent(llm)
        self.risk_analyzer = RiskAnalyzer(llm, project_retriever)
        self.impact_calculator = ImpactCalculator(llm, project_retriever)
        self.web_researcher = WebResearcher(llm)
        self.decision_controller = DecisionController(llm, project_retriever)
        self.report_writer = ReportWriter(llm, project_retriever)
        
    def process_situation(self, user_input: str) -> Dict[str, Any]:
        print("\n" + "="*60)
        print("PROJECT MANAGEMENT ASSISTANT - SITUATION ANALYSIS")
        print("="*60)
        
        # Step 1: Observer
        situation_data = self.observer.process(user_input)
        
        # Step 2: Risk Analysis
        risk_analysis = self.risk_analyzer.analyze(situation_data)
        
        # Step 3: Impact Assessment
        impact_assessment = self.impact_calculator.calculate(situation_data, risk_analysis)
        
        # Step 4: Web Research (if needed)
        web_research = None
        if impact_assessment.get('needs_web_research', False):
            query = impact_assessment.get('web_research_query', 'rental truck prices')
            web_research = self.web_researcher.research(query)
        
        # Step 5: Decision
        decision = self.decision_controller.decide(
            situation_data, risk_analysis, impact_assessment, web_research
        )
        
        # Step 6: Report Writing
        all_data = {
            "situation": situation_data,
            "risk": risk_analysis,
            "impact": impact_assessment,
            "web": web_research,
            "decision": decision
        }
        
        report = self.report_writer.write_report(decision, all_data)
        
        print("\n" + "="*60)
        print("FINAL REPORT")
        print("="*60)
        
        return {
            "email": report,
            "decision": decision,
            "full_analysis": all_data
        }

In [26]:
def clean_json_response(response: str) -> str:
    """
    Clean LLM response to extract valid JSON.
    Handles cases where JSON is wrapped in markdown code blocks.
    """
    # Remove markdown code block formatting
    if '```json' in response:
        # Extract content between ```json and ```
        match = re.search(r'```json\s*(.*?)\s*```', response, re.DOTALL)
        if match:
            return match.group(1).strip()
    elif '```' in response:
        # Extract content between ``` and ```
        match = re.search(r'```\s*(.*?)\s*```', response, re.DOTALL)
        if match:
            return match.group(1).strip()
    
    # If no code blocks, return as is
    return response.strip()

In [27]:
import re
# Initialize the MAS
pm_mas = ProjectManagementMAS(llm, project_retriever)

# Test examples
print("\n=== PROJECT MANAGEMENT MULTI-AGENT SYSTEM READY ===")
print("\nExample usage:")
print('result = pm_mas.process_situation("Jan 30, 2025 - Brother\'s truck is broken, and won\'t be repaired for 6 weeks.")')

# Function for easy testing
def test_situation(situation: str):
    """Test a situation and display results"""
    result = pm_mas.process_situation(situation)
    
    print("\n" + "-"*40)
    print("EMAIL OUTPUT:")
    print("-"*40)
    print(f"To: {', '.join(result['email'].get('to', []))}")
    print(f"Subject: {result['email'].get('subject', 'No subject')}")
    print(f"\n{result['email'].get('body', 'No body')}")
    
    return result

# Run test examples
test_situations = [
    "Jan 22, 2025 - Brother's truck is broken, and won't be repaired for 6 weeks.",
    "Feb 14, 2025 - Rainy day, but should be sunshine again in 2 days time and the forecast shows sunny all next week.",
    "Feb 21, 2025 - New Tarriffs came into effect. Everything is 10% more expensive.",
    "Feb 22, 2025 - Removing fence panels only took 1 hour, not 10."
]

# Uncomment to run tests
for situation in test_situations:
    test_situation(situation)
    print("\n" + "="*80 + "\n")


=== PROJECT MANAGEMENT MULTI-AGENT SYSTEM READY ===

Example usage:
result = pm_mas.process_situation("Jan 30, 2025 - Brother's truck is broken, and won't be repaired for 6 weeks.")

PROJECT MANAGEMENT ASSISTANT - SITUATION ANALYSIS

[Observer Agent] Parsing input...
[Observer Agent] Extracted: Date=2025-01-22, Type=RESOURCE_ISSUE

[Risk Analyzer] Checking against risk register...
[Risk Analyzer] Classification: KNOWN_RISK
[Risk Analyzer] Matched Risk ID: R006

[Impact Calculator] Assessing project impact...
[Impact Calculator] Schedule impact: 42 days
[Impact Calculator] Cost impact: $0
[Impact Calculator] Web research needed: False

[Decision Controller] Evaluating escalation requirements...
[Decision Controller] Escalation required: True
[Decision Controller] Reason: schedule impact exceeds 2 week threshold

[Report Writer] Generating ESCALATION_EMAIL...
[Report Writer] Generated ESCALATION email

FINAL REPORT

----------------------------------------
EMAIL OUTPUT:
----------------